In [82]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import sys

In [83]:
sys.path.append('..')

In [84]:
import modules.transforms

In [86]:
from modules.transforms import *

In [23]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [24]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [25]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [66]:
url='https://docs.google.com/spreadsheets/d/1IjirW9zoW5H-x8AnQcQaKXZXiS1tCuTs0PxSkNxYE0g/export?format=xlsx&gid=0'

In [67]:
r = requests.get(url, allow_redirects=True)
open(path+'cuarentenas.xlsx', 'wb').write(r.content)

46752

In [249]:
cuarentenas=pd.read_excel(path+'cuarentenas.xlsx',index_col=None,nrows=90)

In [250]:
cuarentenas['Area']=cuarentenas['Unnamed: 1'].fillna(method='ffill').astype(str)+' '+cuarentenas['Unnamed: 2'].fillna('').astype(str)

In [251]:
cuarentenas=cuarentenas.set_index('Area').T

In [252]:
cuarentenas=cuarentenas.drop(['Cuarentenas','Unnamed: 2','Unnamed: 1'])

In [253]:
cuarentenas.columns.values[0]='Fecha'

In [254]:
cuarentenas.index=list(range(cuarentenas.shape[0]))

In [255]:
cuarentenas=cuarentenas.melt(id_vars=['Fecha']).rename(columns={'Unnamed: 1':'Area'})

In [256]:
cuarentenas['Fecha']=pd.to_datetime(cuarentenas['Fecha'],errors='coerce').dt.to_pydatetime()

In [257]:
cordones=pd.read_excel(path+'cuarentenas.xlsx',index_col=None,skiprows=93,nrows=20)

In [258]:
cordones=cordones.set_index('Unnamed: 1').T

In [259]:
cordones=cordones.drop(['Unnamed: 2','Unnamed: 0'])
cordones=cordones.reset_index()

In [260]:
cordones=cordones.rename(columns={'index':'Fecha','Valparaíso y Viña del Mar (accesos desde Santiago)':'Valparaíso y Viña del Mar'})

In [261]:
cordones=cordones.melt(id_vars=['Fecha']).rename(columns={'Unnamed: 1':'Area'})

In [262]:
cordones['Fecha']=pd.to_datetime(cordones['Fecha'],errors='coerce').dt.to_pydatetime()

In [263]:
for k in ['cuarentenas']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema cuarentenas exists, will not be created


In [264]:
name='cuarentenas'
schema='cuarentenas'
print("creating table "+name+' ,schema: '+schema)
cuarentenas.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
cuarentenas.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table cuarentenas ,schema: cuarentenas
saving table/data/ETLcache/cuarentenas_29082020_053150.csv in cache


In [265]:
name='cordones'
schema='cuarentenas'
print("creating table "+name+' ,schema: '+schema)
cordones.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
cordones.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table cordones ,schema: cuarentenas
saving table/data/ETLcache/cordones_29082020_053150.csv in cache


In [248]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile.ipynb')

0